# Tutorial. Finite difference methods for scalar conservation laws

In physics, a <i>conservation law</i> states that a particular measurable property of an isolated physical system does not change as the system evolves over time. Exact conservation laws include conservation of mass and energy, conservation of linear momentum, conservation of angular momentum, and conservation of electric charge. It is usually expressed mathematically as a partial differential equation which gives a relation between the amount of the quantity and the "transport" of that quantity. It states that the amount of the conserved quantity at a point or within a volume can only change by the amount of the quantity which flows in or out of the volume.

In this notebook, we are interested in the numerical solution of scalar conservation laws in one space dimension, using [finite difference methods](https://en.wikipedia.org/wiki/Finite_difference_method).

The <tt>numpy</tt> and <tt>matplotlib</tt> packages will be needed.

In [ ]:
import numpy as np

# To draw matplotlib plots within this notebook.
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

## Exercise 1. A linear problem.
We are interested in solving the following Cauchy problem, comprised of a scalar conservation law
$$
\frac{\partial u}{\partial t}(t,x)+\frac{\partial f(u)}{\partial x}(t,x)=0,\ t>0,\ x\in\mathbb{R},
$$
where $f(u)=c\,u$ (such a choice corresponds to a linear transport equation, the scalar $c$ being the constant advection velocity), and of an initial condition
$$
u(0,x)=u_0(x),\ x\in\mathbb{R},
$$
the function $u_0$ being given.

**Question.** Solve this problem using the [method of characteristics](https://en.wikipedia.org/wiki/Method_of_characteristics) and then implement a function returning the value of the solution $u$ at a given time $t$ and a given point $x$.

**Answer.** The characteristic curves of the equation are the parallel half-lines $(x(t),t)$ defined by
$$
x(t)=c\,t+x_0,\ t\in[0,+\infty),\ x_0\in\mathbb{R},
$$
and such that the solution $u$ is constant along them. The solution is then given by
$$
u(t,x)=u_0(x-c\,t),\ t\in[0,+\infty),\ x\in\mathbb{R}.
$$

In [ ]:
def solution(t,x): return u0(x-c*t)

In what follows, we assume that the initial datum $u_0$ is a periodic function. In such a situation, the solution $u$ is also a periodic function in the space variable, which allows to restrict the numerical solution of the problem to a bounded subset $[0,T]\times[0,L]$ of $\mathbb{R}_+\times\mathbb{R}$, where the real number $L$ is a period of the function $u_0$, by adding to the preceding equations the following periodicity condition
$$
u(t,0)=u(t,L),\ t\in[0,T].
$$

In the present case, we set $T=3$, $L=4$ and $c=1$, the restriction of the initial datum to $[0,L]$ being
$$
u_0(x)=\begin{cases}1&\text{if }1\leq x\leq 2,\\0&\text{else.}\end{cases}
$$

In [ ]:
T,L,c=3.,4.,1
def u0(x): return (x%4>=1.) & (x%4<=2.)

In order to compute a numerical approximation to the solution of the problem by the finite difference method, we first define a uniform Cartesian discretisation grid of the domain $[0,T]\times[0,L]$ and the quotient $\lambda=\frac{\Delta t}{\Delta x}$, $\Delta t$ and $\Delta x$ being the lengths of the time and space steps, respectively.

The numerical approximation of the solution to the problem will then be obtained by using successively:

- the forward in time centered in space scheme ([FTCS](http://en.wikipedia.org/wiki/FTCS_scheme)),
$$
u^{n+1}_j=u^n_j-c\lambda\,\frac{u^n_{j+1}-u^n_{j-1}}{2},\ n\in\mathbb{N},\ j\in\mathbb{Z},
$$
- the scheme of the [Lax-Friedrichs method](http://en.wikipedia.org/wiki/Lax%E2%80%93Friedrichs_method),
$$
u^{n+1}_j=\frac{(1-c\lambda)\,u^n_{j+1}+(1+c\lambda)\,u^n_{j-1}}{2},\ n\in\mathbb{N},\ j\in\mathbb{Z},
$$
- the scheme of the [Lax-Wendroff method](http://en.wikipedia.org/wiki/Lax%E2%80%93Wendroff_method),
$$
u^{n+1}_j=u^n_j-c\lambda\,\frac{u^n_{j+1}-u^n_{j-1}}{2}+(c\lambda)^2\,\frac{u^n_{j+1}-2\,u^n_j+u^n_{j-1}}{2},\ n\in\mathbb{N},\ j\in\mathbb{Z}.
$$

We recall that each of the three-point schemes given above can be written in *conservative form*, that is
$$
u^{n+1}_j=u^n_j-\lambda\left(h(u^n_j,u^n_{j+1})-h(u^n_{j-1},u^n_j)\right),\ n\in\mathbb{N},\ j\in\mathbb{Z},
$$
where the function $h$ is the numerical flux associated with the scheme, defined up to an additive constant.

**Question.** Determine the numerical fluxes associated with the conservative form of the above schemes and implement their corresponding functions.

**Answer.** Identifying with the formulas defining the schemes with the conservative form given above, one finds that
$$
h_{FTCS}(u,v)=\frac{c}{2}(u+v),
$$
$$
h_{LF}(u,v)=\frac{1}{2}\left(c(u+v)-\frac{v-u}{\lambda}\right),
$$
and
$$
h_{LW}(u,v)=\frac{c}{2}\left(u+v-c\lambda(v-u)\right).
$$

In [ ]:
# numerical flux for the FTCS scheme
def h_ftcs(u,v): return 0.5*c*(u+v)
# numerical flux for the Lax-Friedrichs scheme
def h_lf(u,v): return 0.5*(c*(u+v)-(v-u)/lam)
# numerical flux for the Lax-Wendroff scheme
def h_lw(u,v): return 0.5*c*(u+v-lam*c*(v-u))

**Question.** Compute a numerical approximation to the solution of the problem with each of the three schemes, using $500$ steps in space and setting the Courant number value to $0.9$ in the [Courant-Friedrichs-Lewy condition](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition).

**Answer.** We subdivide the space interval $[0,L]$ into $M$ sub-intervals of equal length $\Delta x=\frac{L}{M}$. To choose a time step, we rely on the CFL condition in which the Courant number is set to $0.9$, meaning that
$$
|c|\lambda=|c|\frac{\Delta t}{\Delta x}=0.9,
$$
that is $\Delta t=\frac{0.9\Delta x}{|c|}$. We then set $N=\left\lceil\frac{T}{\Delta t}\right\rceil$.
Finally, the discrete form of the periodicity condition is
$$
u^n_0=u^n_{M},\ n\in\{0,\dots,N\}.
$$

In [ ]:
# setting of the space step
M=500
dx=L/M
X=np.linspace(0,L,M+1)

# setting of the time step using the fixed value of the Courant number in the CFL condition
alpha=0.9 # value of the Courant number
dt=alpha*dx/abs(c) # recall that the flux in the equation is linear (f(u)=cu)
N=int(np.ceil(T/dt))

lam=dt/dx

# initialisation for each of the three schemes
U_ftcs,U_lf,U_lw=np.zeros([N+1,M+1]),np.zeros([N+1,M+1]),np.zeros([N+1,M+1])
Up,Um=np.zeros(M+1),np.zeros(M+1)
U_ftcs[0,:],U_lf[0,:],U_lw[0,:]=u0(X),u0(X),u0(X)

t=np.zeros(N+1)

# time loop
for n in range(N):
    # FTCS scheme
    Up[0:M],Up[M]=[U_ftcs[n,i] for i in range(1,M+1)],U_ftcs[n,1]
    Um[0],Um[1:M+1]=U_ftcs[n,M-1],[U_ftcs[n,i] for i in range(M)]
    U_ftcs[n+1,:]=U_ftcs[n,:]-lam*(h_ftcs(U_ftcs[n,:],Up)-h_ftcs(Um,U_ftcs[n,:]))
    # Lax-Friedrichs method
    Up[0:M],Up[M]=[U_lf[n,i] for i in range(1,M+1)],U_lf[n,1]
    Um[0],Um[1:M+1]=U_lf[n,M-1],[U_lf[n,i] for i in range(M)]
    U_lf[n+1,:]=U_lf[n,:]-lam*(h_lf(U_lf[n,:],Up)-h_lf(Um,U_lf[n,:]))
    # Lax-Wendroff method
    Up[0:M],Up[M]=[U_lw[n,i] for i in range(1,M+1)],U_lw[n,1]
    Um[0],Um[1:M+1]=U_lw[n,M-1],[U_lw[n,i] for i in range(M)]
    U_lw[n+1,:]=U_lw[n,:]-lam*(h_lw(U_lw[n,:],Up)-h_lw(Um,U_lw[n,:]))
    
    t[n+1]=t[n]+dt

**Question.** Using the code given below, make animations plotting the computed numerical approximation together with the exact solution of the problem. Comment.

In [ ]:
# size of the animation window
fig,ax=plt.subplots()
ax.set_xlim((0,L))
ax.set_ylim((-0.5,1.5))

# plotting of two curves (one for the exact solution, one for the numerical approximation)
line,=ax.plot([],[],lw=2)
line2,=ax.plot([],[],lw=2)

# function to initialise the plots
def init():
    line.set_data([],[])
    line2.set_data([],[])
    return (line,line2,)

# function to build the animation
def animate(i):
    line.set_data(X,solution(t[i],X)) # plotting of the exact solution
    line2.set_data(X,U[i,:]) # plotting of the numerical approximation from the values stored in the array U
    return (line,line2,)

**Answer.** We start with the FTCS scheme.

In [ ]:
U=U_ftcs
anim=animation.FuncAnimation(fig,animate,init_func=init,frames=N,interval=20,blit=True)
HTML(anim.to_jshtml())

We find that this scheme is unstable under the CFL condition.

We continue with the Lax-Friedrichs method.

In [ ]:
U=U_lf
anim=animation.FuncAnimation(fig,animate,init_func=init,frames=N,interval=20,blit=True)
HTML(anim.to_jshtml())

This scheme is stable under the CFL condition and shows signs of numerical dissipation. This is best understood when one rewrites the scheme as a "correction" of the FTCS scheme:
$$
\frac{u^{n+1}_j-u^n_j}{\Delta t}+a\frac{u^n_{j+1}-u^n_{j-1}}{2\Delta x}=\frac{1}{2}\frac{(\Delta x)^2}{\Delta t}\frac{u^n_{j+1}-2u^n_j+u^n_{j-1}}{(\Delta x)^2},
$$
in which a diffusion term, of order one in $\Delta x$ under the CFL condition, appears (and would correspond, using a modified equation approach, to a second-order partial derivative in the space variable).

We end with the Lax-Wendroff method.

In [ ]:
U=U_lw
anim=animation.FuncAnimation(fig,animate,init_func=init,frames=N,interval=20,blit=True)
HTML(anim.to_jshtml())

This scheme is stable under the CFL condition. It is less dissipative than for the Lax-Friedrichs method, but the approximation shows some spurious oscillations, which grow over time, in the vicinity of the discontinuities of the solution, characteric of the [Gibbs phenomenon](https://en.wikipedia.org/wiki/Gibbs_phenomenon). This is typical of schemes which employ a high-order spatial discretisation (the accuracy of the Lax-Wendroff method being of order two) in the presence of discontinuities. In practice, such problem is avoided through the use of [flux limiters](https://en.wikipedia.org/wiki/Flux_limiter) (which render the scheme non-linear even if the flux in the equation is linear).

## Exercise 2. A non-linear problem.
We are interested in solving the Cauchy problem defined in the preceding exercise, but with a flux which now equal to $f(u)=\frac{u^2}{2}$ (this choice corresponds to the inviscid [Burgers equation](https://en.wikipedia.org/wiki/Burgers%27_equation)) and the initial datum
$$
u_0(x)=\begin{cases}1&\text{if }0\leq x\leq 1,\\0&\text{else.}\end{cases}
$$
In what follows, we consider a solution on the domain $[0,3]\times[-1,4]$.

In [ ]:
T=3.
def u0(x): return (x>=0.) & (x<=1.)

**Question.** Determine the unique entropy solution of the problem and implement a function returning the value of this solution at a given time $t$ and a given point $x$.

**Answer.** With such an initial datum, one must distinguish three zones for the characteristic curves since
$$
x(t)=\begin{cases}x_0&\text{if }x_0\leq0,\\t+x_0&\text{if }0<x_0\leq1,\\x_0&\text{if }x_0>1,\end{cases}\ x_0\in\mathbb{R}.
$$
The curves of the second and third region intersect at the initial time $t=0$. The flux in the equation being a strictly convex function, the Lax entropy condition states that a shock is admissible if the value of the solution to its left is stricly larger than the value to its right. The second discontinuity in the initial datum hence produces a shock wave emanating from $x=1$ and travelling along a curve, parametrised by the function $t\mapsto\xi(t)$, at a speed is given by the [Rankine-Hugoniot condition](https://en.wikipedia.org/wiki/Rankine%E2%80%93Hugoniot_conditions):
$$
\xi'(t)=\frac{1}{2},
$$
so that $\xi(t)=\frac{1}{2}\,t+1$. On the other hand, the first discontinuity in the initial datum gives birth to a rarefaction wave, associated to curves $\frac{x}{t}=\text{constant}$ filling the area between the first and second zones without characteristic curves to define the solution. The unique entropy solution to the problem is thus given by
$$
u(t,x)=\begin{cases}0&\text{if }x<0,\\\dfrac{x}{t}&\text{if }0<x<t,\\1&\text{if }t<x<1+\dfrac{t}{2},\\0&\text{if }x>1+\dfrac{t}{2},\end{cases}\text{ for }0<t<t^*,
$$
up to a time $t^*$ sastifying $t^*=1+\frac{t^*}{2}$, that is $t^*=2$, for which the rarefaction wave meets the shock wave. For $t>2$, a discontinuity separates the rarefaction wave on the left from the vacuum (that is, the constant state with value $0$) on the right. Using once again the Rankine-Hugoniot conditions, we find that the function $\xi$ parametrising the discontinuity curse satisfies
$$
\xi'(t)=\frac{1}{2t}\,\xi(t),\ \xi(2)=2,
$$
so that $\xi(t)=\sqrt{2t}$ and finally
$$
u(t,x)=\begin{cases}0&\text{if }x<0,\\\dfrac{x}{t}&\text{if }0<x<\sqrt{2t},\\0&\text{if }x>\sqrt{2t},\end{cases}\text{ for }t>2.
$$

In [ ]:
def solution(t,x):
    if (t==0):
       sol=u0(x)
    elif (t>0) & (t<2):
        sol=np.where([(x>=0)&(x<t)],x/t,np.where([(x>=t)&(x<1.+0.5*t)],np.ones(len(x)),np.zeros(len(x))))
    else:
        sol=np.where([(x>=0)&(x<np.sqrt(2.*t))],x/t,np.zeros(len(x)))
    return sol

The flux $f$ being a nonlinear function, we use the following adaptations for the schemes of the Lax-Friedrichs method 
$$
u_j^{n+1}=\frac{1}{2}\,\left(u_{j+1}^n+u_{j-1}^n\right)-\frac{\lambda}{2}\,\left(f(u_{j+1}^n)-f(u_{j-1}^n)\right),\ n\in\mathbb{N},\ j\in\mathbb{Z},
$$
and of the Lax-Wendroff method
$$
u_j^{n+1}=u_j^n-\frac{\lambda}{2}\,\left(f(u_{j+1}^n)-f(u_{j-1}^n)\right)+\frac{\lambda^2}{2}\left(f'\left(\frac{u_{j+1}^n+u_j^n}{2}\right)\left(f(u_{j+1}^n)-f(u_j^n)\right)-f'\left(\frac{u_j^n+u_{j-1}^n}{2}\right)\left(f(u_j^n)-f(u_{j-1}^n)\right)\right),\ n\in\mathbb{N},\ j\in\mathbb{Z},
$$
to numerically solve the problem.

**Question.** Use these two methods to numerically approximate the entropy solution of the problem. The Courant number will be fixed to the value $0.9$ in the CFL condition.

In [ ]:
# setting of the space step
M=500
dx=5/M
X=np.linspace(-1,4,M+1)

alpha=0.9 # value of the Courant number in the CFL condition

def flux(u): return 0.5*u*u # flux
def dflux(u): return u # derivative of the flux (for the Lax-Wendroff methode)

# numerical flux for the Lax-Friedrichs method
def h_lf(u,v): return 0.5*(flux(v)+flux(u)-(v-u)/lam)
# numerical flux for the Lax-Wendroff method
def h_lw(u,v):
    fu,fv=flux(u),flux(v)
    return 0.5*(fv+fu-lam*dflux(0.5*(u+v))*(fv-fu))

# initialisation for each of the schemes
Nmax=2000
U_lf,U_lw=np.zeros([Nmax,M+1]),np.zeros([Nmax,M+1])
Up,Um=np.zeros(M+1),np.zeros(M+1)
U_lf[0,:],U_lw[0,:]=u0(X),u0(X)

t_lf,t_lw=np.zeros(Nmax+1),np.zeros(Nmax+1)
n_lf,n_lw=0,0

# time loop
# Lax-Friedrichs method
while (t_lf[n_lf]<T):
    # setting of the time step using the CFL condition
    dt=alpha*dx/np.amax(abs(dflux(U_lf)))
    lam=dt/dx

    Up[0:M],Up[M]=[U_lf[n_lf,i] for i in range(1,M+1)],0
    Um[0],Um[1:M+1]=0,[U_lf[n_lf,i] for i in range(M)]
    U_lf[n_lf+1,:]=U_lf[n_lf,:]-lam*(h_lf(U_lf[n_lf,:],Up)-h_lf(Um,U_lf[n_lf,:]))
    
    t_lf[n_lf+1]=t_lf[n_lf]+dt
    n_lf=n_lf+1
    
# Lax-Wendroff method
while (t_lw[n_lw]<T):
    # setting of the time step using the CFL condition
    dt=alpha*dx/np.amax(abs(dflux(U_lw)))
    lam=dt/dx
    
    Up[0:M],Up[M]=[U_lw[n_lw,i] for i in range(1,M+1)],0
    Um[0],Um[1:M+1]=0,[U_lw[n_lw,i] for i in range(M)]
    U_lw[n_lw+1,:]=U_lw[n_lw,:]-lam*(h_lw(U_lw[n_lw,:],Up)-h_lw(Um,U_lw[n_lw,:]))

    t_lw[n_lw+1]=t_lw[n_lw]+dt
    n_lw=n_lw+1

**Question.** Make animations using the code given in the preceding exercise (the size of the animation window will be modified as indicated below). What can be said of the computed numerical approximations?

In [ ]:
# setting of the size of the animation window
ax.set_xlim((-1,4))
ax.set_ylim((-1.,1.5))

t,U=t_lf,U_lf
anim=animation.FuncAnimation(fig,animate,init_func=init,frames=n_lf,interval=10,blit=True)
HTML(anim.to_jshtml())

**Answer.** The Lax-Friedrichs method produces an approximation of the entropy solution.

In [ ]:
t,U=t_lw,U_lw
anim=animation.FuncAnimation(fig,animate,init_func=init,frames=n_lw,interval=10,blit=True)
HTML(anim.to_jshtml())

The numerical approximation produced by the Lax-Wendroff method possesses discontinuities which do not satisfy the entropy condition: the weak solution to which it converges is not an entropy solution of the problem. This is in line with the [Godunov theorem](https://en.wikipedia.org/wiki/Godunov%27s_theorem), which states that a linear numerical scheme for solving partial differential equations having the property of not generating new extrema can be at most first-order accurate.